# Forward Propagation

Here is a neural network that takes two inputs, has one hidden layer with two nodes, and an output layer with one node.

<img src="http://cocl.us/neural_network_example" alt="Neural Network Example" width="600px">

In [2]:
import numpy as np

weights = np.around(np.random.uniform(size=6), decimals=2)
biases = np.around(np.random.uniform(size=3), decimals=2)

print(weights)
print(biases)

[0.65 0.85 0.84 0.97 0.98 0.65]
[0.34 0.63 0.88]


Now that we have the weights and the biases defined for the network, let's compute the output for a given input, $x_1$ and $x_2$.

In [4]:
x_1 = 0.5
x_2 = 0.85

print('x1 is {} amd x2 is {}'.format(x_1, x_2))

x1 is 0.5 amd x2 is 0.85


Let's start by computing the weighted sum of the inputs, $z_{1, 1}$, at the first node of the hidden layer.


In [6]:
z_11 = x_1 * weights[0] + x_2 * weights[1] + biases[0]

print('The weighted sum of the inputs at the first node in the hidden layer is {}'.format(z_11))

The weighted sum of the inputs at the first node in the hidden layer is 1.3875


In [7]:
z_12 = x_1 * weights[2] + x_2 * weights[3] + biases[1]

print('The weighted sum of the inputs at the second node in the hidden layer is {}'.format(z_12))

The weighted sum of the inputs at the second node in the hidden layer is 1.8744999999999998


Next, assuming a sigmoid activation function, let's compute the activation of the nodes 
in the hidden layer.


In [9]:
a_11 = 1.0 / (1.0 + np.exp(-z_11))
a_12 = 1.0 / (1.0 + np.exp(-z_12))

print('The activation of the first node in the hidden layer is {}'.format(np.around(a_11, decimals=4)))
print('The activation of the second node in the hidden layer is {}'.format(np.around(a_12, decimals=4)))

The activation of the first node in the hidden layer is 0.8002
The activation of the second node in the hidden layer is 0.867


In [11]:
z_2 = a_11 * weights[4] + a_12 * weights[5] + biases[2]

print('The weighted sum of the inputs at the node in the output layer is {}'.format(np.around(z_2, decimals=4)))

The weighted sum of the inputs at the node in the output layer is 2.2277


In [13]:
a_2 = 1.0 / (1.0 + np.exp(-z_2))

print('The output of the network for x1 = 0.5 and x2 = 0.85 is {}'.format(np.around(a_2, decimals=4)))

The output of the network for x1 = 0.5 and x2 = 0.85 is 0.9027


# Build a Neural Network

In order to code an automatic way of making predictions, let's generalize our network. A general network would take $n$ inputs, would have many hidden layers, each hidden layer having $m$ nodes, and would have an output layer. Although the network is showing one hidden layer, but we will code the network to have many hidden layers. Similarly, although the network shows an output layer with one node, we will code the network to have more than one node in the output layer.

<img src="http://cocl.us/general_neural_network" alt="Neural Network General" width="600px">


Let's start by formally defining the structure of the network.

In [14]:
n = 2 
num_hidden_layers = 2
m = [2, 2] # number of nodes in each hidden layer
num_nodes_output = 1 # number of nodes in the output layer


Now that we defined the structure of the network, let's go ahead and initialize the weights and the biases in the network to random numbers.

In [17]:
def initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output):
  num_nodes_previous = num_inputs
  network = {}

  for layer in range(num_hidden_layers + 1):
    # determine name of layer
    if layer == num_hidden_layers:
      layer_name = 'output'
      num_nodes = num_nodes_output
    else:
      layer_name = 'layer_{}'.format(layer + 1)
      num_nodes = num_nodes_hidden[layer]

    # initialize weights and biases associated with each node in the current layer
    network[layer_name] = {}
    for node in range(num_nodes):
      node_name = 'node_{}'.format(node + 1)
      network[layer_name][node_name] = {
        'weights': np.around(np.random.uniform(size=num_nodes_previous), decimals=2),
        'bias': np.around(np.random.uniform(size=1), decimals=2) 
      }

    num_nodes_previous = num_nodes

  return network
  

#### Use the *initialize_network* function to create a network that:

1. takes 5 inputs
2. has three hidden layers
3. has 3 nodes in the first layer, 2 nodes in the second layer, and 3 nodes in the third layer
4. has 1 node in the output layer

Call the network **small_network**.


In [18]:
num_inputs = 5
num_hidden_layers = 3
num_nodes_hidden = [3, 2, 3]
num_nodes_output = 1
small_network = initialize_network(num_inputs, num_hidden_layers, num_nodes_hidden, num_nodes_output)
print(small_network)

{'layer_1': {'node_1': {'weights': array([0.88, 0.66, 0.03, 0.9 , 0.84]), 'bias': array([0.74])}, 'node_2': {'weights': array([0.94, 0.76, 0.26, 0.08, 0.49]), 'bias': array([0.26])}, 'node_3': {'weights': array([0.71, 0.5 , 0.22, 0.44, 0.35]), 'bias': array([0.95])}}, 'layer_2': {'node_1': {'weights': array([0.51, 0.81, 0.3 ]), 'bias': array([0.55])}, 'node_2': {'weights': array([0.55, 0.31, 0.45]), 'bias': array([0.09])}}, 'layer_3': {'node_1': {'weights': array([0.63, 0.23]), 'bias': array([0.61])}, 'node_2': {'weights': array([0.09, 0.91]), 'bias': array([0.03])}, 'node_3': {'weights': array([0.23, 0.93]), 'bias': array([0.82])}}, 'output': {'node_1': {'weights': array([0.73, 0.74, 0.31]), 'bias': array([0.24])}}}


## Compute Weighted Sum at Each Node


The weighted sum at each node is computed as the dot product of the inputs and the weights plus the bias. So let's create a function called compute_weighted_sum that does just that.

In [19]:
def compute_weighted_sum(inputs, weights, bias):
  return np.sum(inputs * weights) + bias

Let's generate 5 inputs that we can feed to **small_network**.


In [20]:
from random import seed

np.random.seed(12)
inputs = np.around(np.random.uniform(size=5), decimals=2)

print('The inputs to the network are {}'.format(inputs))

The inputs to the network are [0.15 0.74 0.26 0.53 0.01]


In [23]:
node_weights = small_network['layer_1']['node_1']['weights']
node_bias = small_network['layer_1']['node_1']['bias']

weighted_sum = compute_weighted_sum(inputs, node_weights, node_bias)
print(f'The weighted sum at the first node in the hidden layer is {np.around(weighted_sum[0], decimals=4)}')

The weighted sum at the first node in the hidden layer is 1.8536


## Compute Node Activation

Recall that the output of each node is simply a non-linear tranformation of the weighted sum. We use activation functions for this mapping. Let's use the sigmoid function as the activation function here. So let's define a function that takes a weighted sum as input and returns the non-linear transformation of the input using the sigmoid function.

In [24]:
def node_activation(weighted_sum):
  return 1.0 / (1.0 + np.exp(-1 * weighted_sum))

In [25]:
node_output = node_activation(weighted_sum)
print(f'The output of the first node in the hidden layer is {np.around(node_output[0], decimals=4)}')

The output of the first node in the hidden layer is 0.8645


## Forward Propagation

The final piece of building a neural network that can perform predictions is to put everything together. So let's create a function that applies the *compute_weighted_sum* and *node_activation* functions to each node in the network and propagates the data all the way to the output layer and outputs a prediction for each node in the output layer.

The way we are going to accomplish this is through the following procedure:

1. Start with the input layer as the input to the first hidden layer.
2. Compute the weighted sum at the nodes of the current layer.
3. Compute the output of the nodes of the current layer.
4. Set the output of the current layer to be the input to the next layer.
5. Move to the next layer in the network.
6. Repeat steps 2 - 5 until we compute the output of the output layer.


In [26]:
def forward_propagate(network, inputs):
  layer_inputs = list(inputs) # start with the input layer as the input to the first hidden layer

  for layer in network:
    layer_data = network[layer]

    layer_outputs = []
    for layer_node in layer_data:
      node_data = layer_data[layer_node]

      # compute the weighted sum and the output of each node at the same time 
      node_output = node_activation(compute_weighted_sum(layer_inputs, node_data['weights'], node_data['bias']))
      layer_outputs.append(np.around(node_output[0], decimals=4))

    if layer != 'output':
      print(f'The outputs of the nodes in hidden layer number {layer.split('_')[1]} is {layer_outputs}')

    layer_inputs = layer_outputs #the output of this layer to be the input to next layer

  network_predictions = layer_outputs
  return network_predictions

In [27]:
predictions = forward_propagate(small_network, inputs)
print(f'The predicted value by the network for the given input is {np.around(predictions[0], decimals=4)}')

The outputs of the nodes in hidden layer number 1 is [np.float64(0.8645), np.float64(0.7462), np.float64(0.8482)]
The outputs of the nodes in hidden layer number 2 is [np.float64(0.8641), np.float64(0.7647)]
The outputs of the nodes in hidden layer number 3 is [np.float64(0.7909), np.float64(0.6908), np.float64(0.8494)]
The predicted value by the network for the given input is 0.8309


So we built the code to define a neural network. We can specify the number of inputs that a neural network can take, the number of hidden layers as well as the number of nodes in each hidden layer, and the number of nodes in the output layer.


In [28]:
my_network = initialize_network(5, 3, [2, 3, 2], 3)
inputs = np.around(np.random.uniform(size=5), decimals=2)
predictions = forward_propagate(my_network, inputs)
print(f'The predicted values by the network for the given input are {predictions}')

The outputs of the nodes in hidden layer number 1 is [np.float64(0.8857), np.float64(0.8889)]
The outputs of the nodes in hidden layer number 2 is [np.float64(0.7822), np.float64(0.6965), np.float64(0.7411)]
The outputs of the nodes in hidden layer number 3 is [np.float64(0.868), np.float64(0.881)]
The predicted values by the network for the given input are [np.float64(0.8952), np.float64(0.8222), np.float64(0.8035)]
